In [1]:
import pandas as pd
from train_test import train_test

In [2]:
df = pd.read_csv("../data/data_scaled/DF_DEMANDA_10_25_PROCESADO.csv")
df

,año,valor_(GWh),dia_semana_sin,dia_semana_cos,dia_sin,dia_cos,mes_sin,mes_cos,fecha
0,0.0,0.267689,-0.974928,-0.222521,0.201299,9.795299e-01,0.0,1.000000,2011-01-01
1,0.0,0.349814,-0.781831,0.623490,0.394356,9.189578e-01,0.0,1.000000,2011-01-02
2,0.0,0.714860,0.000000,1.000000,0.571268,8.207634e-01,0.0,1.000000,2011-01-03
3,0.0,0.788024,0.781831,0.623490,0.724793,6.889669e-01,0.0,1.000000,2011-01-04
4,0.0,0.719847,0.974928,-0.222521,0.848644,5.289640e-01,0.0,1.000000,2011-01-05
...,...,...,...,...,...,...,...,...,...
5165,1.0,0.536621,-0.433884,-0.900969,-1.000000,-1.836970e-16,0.5,0.866025,2025-02-21
5166,1.0,0.341652,-0.974928,-0.222521,-0.974928,2.225209e-01,0.5,0.866025,2025-02-22
5167,1.0,0.218642,-0.781831,0.623490,-0.900969,4.338837e-01,0.5,0.866025,2025-02-23
5168,1.0,0.486300,0.000000,1.000000,-0.781831,6.234898e-01,0.5,0.866025,2025-02-24


In [ ]:
x_train, x_test, y_train, y_test = train_test(df)

In [4]:
x_train

,año,dia_semana_sin,dia_semana_cos,dia_sin,dia_cos,mes_sin,mes_cos
0,0.000000,-0.974928,-0.222521,0.201299,0.979530,0.0,1.000000e+00
1,0.000000,-0.781831,0.623490,0.394356,0.918958,0.0,1.000000e+00
2,0.000000,0.000000,1.000000,0.571268,0.820763,0.0,1.000000e+00
3,0.000000,0.781831,0.623490,0.724793,0.688967,0.0,1.000000e+00
4,0.000000,0.974928,-0.222521,0.848644,0.528964,0.0,1.000000e+00
...,...,...,...,...,...,...,...
4131,0.785714,-0.781831,0.623490,-0.951057,0.309017,1.0,6.123234e-17
4132,0.785714,0.000000,1.000000,-0.866025,0.500000,1.0,6.123234e-17
4133,0.785714,0.781831,0.623490,-0.743145,0.669131,1.0,6.123234e-17
4134,0.785714,0.974928,-0.222521,-0.587785,0.809017,1.0,6.123234e-17


In [5]:
x_test

,año,dia_semana_sin,dia_semana_cos,dia_sin,dia_cos,mes_sin,mes_cos
4136,0.785714,-0.433884,-0.900969,-2.079117e-01,9.781476e-01,1.000000,6.123234e-17
4137,0.785714,-0.974928,-0.222521,-2.449294e-16,1.000000e+00,1.000000,6.123234e-17
4138,0.785714,-0.781831,0.623490,2.012985e-01,9.795299e-01,0.866025,-5.000000e-01
4139,0.785714,0.000000,1.000000,3.943559e-01,9.189578e-01,0.866025,-5.000000e-01
4140,0.785714,0.781831,0.623490,5.712682e-01,8.207634e-01,0.866025,-5.000000e-01
...,...,...,...,...,...,...,...
5165,1.000000,-0.433884,-0.900969,-1.000000e+00,-1.836970e-16,0.500000,8.660254e-01
5166,1.000000,-0.974928,-0.222521,-9.749279e-01,2.225209e-01,0.500000,8.660254e-01
5167,1.000000,-0.781831,0.623490,-9.009689e-01,4.338837e-01,0.500000,8.660254e-01
5168,1.000000,0.000000,1.000000,-7.818315e-01,6.234898e-01,0.500000,8.660254e-01


In [6]:
y_train

0       0.267689
1       0.349814
2       0.714860
3       0.788024
4       0.719847
          ...   
4131    0.189863
4132    0.379254
4133    0.431515
4134    0.454577
4135    0.456630
Name: valor_(GWh), Length: 4136, dtype: float64

In [7]:
y_test

4136    0.412012
4137    0.237042
4138    0.090032
4139    0.286354
4140    0.431176
          ...   
5165    0.536621
5166    0.341652
5167    0.218642
5168    0.486300
5169    0.550413
Name: valor_(GWh), Length: 1034, dtype: float64